In [3]:
import nltk
import pandas as pd
import string
import spacy
from spacy.lang.lt.stop_words import STOP_WORDS
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy import mean
from sklearn.model_selection import RepeatedStratifiedKFold

def text_processing(text):
    # 1. Skyrybos ženklų šalinimas
    nopunct = [char for char in text if char not in string.punctuation]
    nopunct = ''.join(nopunct)
    nopunct = ' '.join(nopunct.split())
    #Lemavimas
    doc = nlp(nopunct)
    string1 = ''
    for word in doc:
        string1 += word.lemma_ + ' '
    string1 = string1[:-1]
    # 3. "Stop" žodžių šalinimas
    doc = nlp(string1)
    return [word.text.lower() for word in doc if not word.is_stop]


def text_preprocessor(text):
    nopunct = [char for char in text if char not in string.punctuation]
    nopunct = ''.join(nopunct)
    nopunct = ' '.join(nopunct.split())

    # Lemmatising
    doc = nlp(nopunct)
    string1 = ''
    for word in doc:
        string1 += word.lemma_ + ' '
    return string1


def label_encoding(data):
    y = data['Author']
    labelencoder = LabelEncoder()
    y = labelencoder.fit_transform(y)
    return y


def word_cloud_visualisation(data, X):
    wordcloud1 = WordCloud().generate(X[1])  # Biliunas
    wordcloud2 = WordCloud().generate(X[900])  # Zemaite
    wordcloud3 = WordCloud().generate(X[301])  # Donelaitis

    print(X[1])
    print(data['Author'][1])
    plt.imshow(wordcloud1, interpolation='bilinear')
    plt.show()
    print(X[900])
    print(data['Author'][900])
    plt.imshow(wordcloud2, interpolation='bilinear')
    plt.show()
    print(X[301])
    print(data['Author'][301])
    plt.imshow(wordcloud3, interpolation='bilinear')
    plt.show()


def model_trainning(text_bow_train, y_train, classifier):
    model = classifier
    model = model.fit(text_bow_train, y_train)
    return model


def show_confusion_matrix(y_test, predictions):
    matrix = pd.crosstab(y_test, predictions)
    # matrix = confusion_matrix(y_test,predictions)
    norm_matrix = matrix / matrix.sum(axis=1)[:, np.newaxis]
    print(norm_matrix)
    plt.matshow(norm_matrix, cmap=plt.cm.Reds)
    plt.colorbar()
    plt.ylabel("Actual")
    plt.xlabel("Predicted")


# ngram = (1,1)
spacy_stopwords = spacy.lang.lt.stop_words.STOP_WORDS
data = pd.read_csv(f'./spacy.csv')
nlp = spacy.load('lt_core_news_sm')


X = data['Text']
y = label_encoding(data)

# 
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1)



new_X_train = []
new_y_train = []


def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens



train_documents = []
test_documents = []
i = 0

n_words= input("Enter n_words value: ")
n_words = int(n_words)

new_X_train = []
new_y_train = []
count = 0;
# n_words = 10000
for text, author in zip(X, y):
    words = text.split()
    grouped_words = [' '.join(words[i: i + n_words]) for i in range(0, len(words), n_words)]
    new_X_train.extend(grouped_words)
    new_y_train.extend([author] * len(grouped_words))
    
new_X_train=X;
new_y_train=y;

with open("train_data.txt", "w", encoding="utf-8") as f:
     for text in new_X_train:
         f.write(text)
         f.write("\n")

new_X_train1 = []
new_y_train1 = []
newText = ""
# count - simbolių n-gramų ilgis
count=5;
index = 0;
j=0;
i=0;
letterIndex = 0;
x =0


for text in new_X_train:
    for letter in text:
        if index < count and letter !=" ":
            newText = newText + letter;
            index=index+1
            letterIndex= letterIndex +1;
        elif letter !=" " and letterIndex<len(text):
            newText = newText + " "
            for i in range((count-1),-1,-1):
                newText = newText + text[letterIndex-i]

            # index = count-1;
            index= index+1;
            letterIndex= letterIndex +1;
        elif letter == " ":
            newText = newText + " "
            index = 0
            letterIndex= letterIndex +1;  



    new_X_train1.append(newText); 
    letterIndex = 0;
    newText="";
    index=0;
    i=0;
    x=x+1



X = pd.Series(new_X_train1)
y = np.array(new_y_train, dtype=int)

# Savybiu gavyba:
spacy_stopwords = spacy.lang.lt.stop_words.STOP_WORDS
# vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=spacy_stopwords)
vectorizer = TfidfVectorizer(stop_words=spacy_stopwords)
X = vectorizer.fit_transform(X,y)
# Kryzmine validacija:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=30)

# Klasifikacija:
for classifier in ('svc','multinomial','logistic_regression'):

    if classifier == 'multinomial':
                    print("Multinomial NB")
                    classifier = MultinomialNB()
                    scores = cross_val_score(classifier, X, y, scoring='accuracy', n_jobs=1, cv=cv)
                    print(mean(scores))
                    
    elif classifier == 'svc':
                    print("SVC")
                    classifier = LinearSVC()                    
                    scores = cross_val_score(classifier, X, y, scoring='accuracy', n_jobs=1, cv=cv)            
                    print(mean(scores))                 
                 
    elif classifier == 'logistic_regression':
                    print("Logistic regression")
                    model1 = LogisticRegression(max_iter=1000)
                    scores = cross_val_score(model1, X, y, cv=cv)
                    print(mean(scores))
        
                # print(f"Parametrai: lemuoklis = {m}, savybių gavyba = {v}, n-gramos = {ng}, klasifikatorius = {cl}")
    

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.